In [1]:
import requests
import pandas as pd

In [3]:
# Base URLs for the API
dataset_id = "GFSMAB2020"
base_url = "http://dataservices.imf.org/REST/SDMX_JSON.svc"

def get_dimensions(dataset_id):
    url = f"{base_url}/DataStructure/{dataset_id}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    dimensions = []
    
    # Inspect the structure of the response
    if "Structure" in data and "KeyFamilies" in data["Structure"]:
        key_family = data["Structure"]["KeyFamilies"]["KeyFamily"]
        
        # Ensure "Components" and "Dimension" exist
        components = key_family.get("Components", {}).get("Dimension", [])
        if isinstance(components, list):
            for component in components:
                dimensions.append({
                    "Dimension": component.get("@codelist", "UnknownCodelist"),
                    "Name": component.get("@id", "UnknownID")
                })
        else:
            print("Unexpected structure for dimensions:", components)
    
    return dimensions


# Function to retrieve possible values for a given dimension
def get_dimension_values(codelist_id):
    url = f"{base_url}/CodeList/{codelist_id}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    values = []
    if "Structure" in data and "CodeLists" in data["Structure"]:
        codes = data["Structure"]["CodeLists"]["CodeList"]["Code"]
        for code in codes:
            values.append({
                "Code": code["@value"],
                "Description": code["Description"]["#text"]
            })
    return values

# Retrieve dimensions for the dataset
print("Fetching dimensions...")
dimensions = get_dimensions(dataset_id)

# Fetch possible values for each dimension
all_dimension_values = {}
for dimension in dimensions:
    print(f"Fetching values for dimension: {dimension['Name']}")
    values = get_dimension_values(dimension["Dimension"])
    all_dimension_values[dimension["Name"]] = values

# Save the dimensions and their values to a DataFrame
output = []
for dimension, values in all_dimension_values.items():
    for value in values:
        output.append({
            "Dimension": dimension,
            "Code": value["Code"],
            "Description": value["Description"]
        })

df = pd.DataFrame(output)
df.to_csv("dimensions_and_values.csv", index=False)
print("Dimensions and values saved to 'dimensions_and_values.csv'")


Fetching dimensions...
Fetching values for dimension: UnknownID
Fetching values for dimension: UnknownID
Fetching values for dimension: UnknownID
Fetching values for dimension: UnknownID
Fetching values for dimension: UnknownID
Dimensions and values saved to 'dimensions_and_values.csv'


In [ ]:
import requests
import pandas as pd

# 1. Define your base URL and dataset
BASE_URL = "http://dataservices.imf.org/REST/SDMX_JSON.svc/"
DATASET_ID = "GFSMAB2017"

# 2. Use wildcards for area, sector, unit, and indicator (and 'A' for annual frequency)
#    Key format: "CompactData/<DATASET_ID>/A.*.*.*.*"
wildcard_key = f"CompactData/{DATASET_ID}/A.*.*.*.*"

# 3. Make the request
print(f"Requesting data with key: {wildcard_key}")
response = requests.get(f"{BASE_URL}{wildcard_key}")
if response.status_code != 200:
    print("Server error or invalid request:", response.status_code, response.text)
    # Exit or handle error as needed
    exit()

# 4. Parse JSON
data_json = response.json()

# 5. Navigate to the Series
data_set = data_json.get("CompactData", {}).get("DataSet")
series = data_set.get("Series") if data_set else None

# If there's no data, exit early
if not series:
    print("No data found for wildcard query.")
    exit()

# 6. Ensure series is a list
if isinstance(series, dict):
    series = [series]

# Prepare a list to store all observations
results = []

# 7. Iterate over each series in the data
for s in series:
    # Each series has dimension attributes like @FREQ, @REF_AREA, @REF_SECTOR, etc.
    freq = s.get("@FREQ")
    ref_area = s.get("@REF_AREA")
    sector = s.get("@REF_SECTOR")
    unit = s.get("@UNIT_MEASURE")
    indicator = s.get("@CLASSIFICATION")

    # 8. Observations are in s["Obs"], which can be a list or a single dict
    obs_list = s.get("Obs", [])
    if isinstance(obs_list, dict):
        obs_list = [obs_list]

    for obs in obs_list:
        time_period = obs.get("@TIME_PERIOD")
        value = obs.get("@OBS_VALUE")

        # Only store non-missing values
        if value is not None:
            results.append({
                "FREQ": freq,
                "AREA_CODE": ref_area,
                "SECTOR_CODE": sector,
                "UNIT_CODE": unit,
                "INDICATOR_CODE": indicator,
                "TIME_PERIOD": time_period,
                "VALUE": value
            })

# 9. Convert to a DataFrame
df = pd.DataFrame(results)
print(f"Retrieved {len(df)} rows from the wildcard request.")

# 10. Save to CSV
output_file = "gfsmab2017_wildcard.csv"
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")


Requesting data with key: CompactData/GFSMAB2017/A.*.*.*.*
No data found for wildcard query.


TypeError: 'NoneType' object is not iterable

: 

http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database ID}/{frequency}.{item1 from
dimension1}+{item2 from dimension1}+{item N from dimension1}.{item1 from
dimension2}+{item2 from dimension2}+{item M from dimension2}?startPeriod={start
date}&endPeriod={end date}

In [2]:
import requests
import time

# Base URL for IMF API
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

# Dataset details
dataset = 'GFSMAB2017'
indicator = 'G63_FD4'  # Example: Gross debt (D4) at market value
sector = 'S1311B'      # Example: Budgetary central government
unit = 'XDC'           # Domestic currency
frequency = 'A'        # Annual data

# Function to fetch geographical areas
def get_countries():
    codelist_key = f'CodeList/CL_AREA_{dataset}'
    response = requests.get(f'{url}{codelist_key}')
    response.raise_for_status()
    data = response.json()
    return [
        code['@value'] for code in 
        data['Structure']['CodeLists']['CodeList']['Code']
    ]

# Function to fetch data for a specific country
def fetch_data(country):
    key = f'CompactData/{dataset}/{frequency}.{country}.{sector}.{unit}.{indicator}'
    response = requests.get(f'{url}{key}')
    if response.status_code == 200:
        try:
            data = response.json()
            series = data.get('CompactData', {}).get('DataSet', {}).get('Series', {})
            return {
                'country': country,
                'data': series.get('Obs', [])
            }
        except Exception as e:
            return {'country': country, 'error': str(e)}
    else:
        return {'country': country, 'error': f"HTTP {response.status_code}"}

# Fetch list of countries
countries = get_countries()
print(f"Retrieved {len(countries)} countries.")

# Loop through countries and fetch data
results = []
for country in countries:
    print(f"Fetching data for {country}...")
    result = fetch_data(country)
    results.append(result)
    time.sleep(0.5)  # To avoid overwhelming the server

# Process results
for result in results:
    if 'error' in result:
        print(f"Error fetching data for {result['country']}: {result['error']}")
    else:
        print(f"Data for {result['country']}: {result['data']}")

# Example: Save results to a file
import json
with open('imf_data.json', 'w') as f:
    json.dump(results, f, indent=4)
print("Results saved to 'imf_data.json'.")



Retrieved 219 countries.
Fetching data for AF...
Fetching data for AL...
Fetching data for DZ...
Fetching data for AO...
Fetching data for AI...
Fetching data for AG...
Fetching data for AR...
Fetching data for AM...
Fetching data for AW...
Fetching data for AU...
Fetching data for AT...
Fetching data for AZ...
Fetching data for BS...
Fetching data for BH...
Fetching data for BD...
Fetching data for BB...
Fetching data for BY...
Fetching data for BE...
Fetching data for BZ...
Fetching data for BJ...
Fetching data for BT...
Fetching data for BO...
Fetching data for BA...
Fetching data for BW...
Fetching data for BR...
Fetching data for BN...
Fetching data for BG...
Fetching data for BF...
Fetching data for BI...
Fetching data for CV...
Fetching data for KH...
Fetching data for CM...
Fetching data for CA...
Fetching data for KY...
Fetching data for CF...
Fetching data for TD...
Fetching data for CL...
Fetching data for HK...
Fetching data for MO...
Fetching data for CN...
Fetching data f

In [ ]:
import requests
import pandas as pd
import time  # To enable pauses between requests

# Base URL
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/"
dataset_id = "GFSMAB2017"

###############################################################################
# 1. Retrieve codelists for countries, sectors, and units
###############################################################################
def fetch_codelist(codelist_id):
    """Fetches and returns a list of (code, description) tuples."""
    response = requests.get(f"{url}CodeList/{codelist_id}")
    response.raise_for_status()
    data = response.json()
    # Navigate to the Code section
    codes = data["Structure"]["CodeLists"]["CodeList"]["Code"]
    return [(c["@value"], c["Description"]["#text"]) for c in codes]

try:
    area_codes = fetch_codelist("CL_AREA_GFSMAB2017")
    sector_codes = fetch_codelist("CL_SECTOR_GFSMAB2017")
    unit_codes = fetch_codelist("CL_UNIT_GFSMAB2017")
except Exception as e:
    print(f"Error fetching codelists: {e}")
    raise SystemExit

# Print how many codes were retrieved
print(f"Fetched {len(area_codes)} area codes.")
print(f"Fetched {len(sector_codes)} sector codes.")
print(f"Fetched {len(unit_codes)} unit codes.")

###############################################################################
# 2. Automated Loop to Retrieve Data
###############################################################################
results = []

# We'll use annual frequency ('A') for the FREQ dimension
# The dimension order (based on the dataset structure) is:
#   1) FREQ  2) AREA  3) SECTOR  4) UNIT  5) INDICATOR
# Final key format: "CompactData/GFSMAB2017/A.<AREA>.<SECTOR>.<UNIT>.<INDIC>"
for area_val, area_desc in area_codes:
    for sector_val, sector_desc in sector_codes:
        for unit_val, unit_desc in unit_codes:
            for indicator in indicators:
                key = (f"CompactData/{dataset_id}/A."
                       f"{area_val}.{sector_val}.{unit_val}.{indicator}")

                # Pause briefly between requests
                time.sleep(1)

                try:
                    resp = requests.get(f"{url}{key}")
                    resp.raise_for_status()
                    data_json = resp.json()
                except Exception as e:
                    # You might want to log failures or skip them
                    print(f"Error retrieving {key}: {e}")
                    continue

                # Parse the time-series data (if present)
                # Typical structure:
                # data_json["CompactData"]["DataSet"]["Series"][ ... ]["Obs"]
                try:
                    series_data = data_json["CompactData"]["DataSet"].get("Series")
                    # If there's no 'Series', skip
                    if not series_data:
                        continue

                    # 'Series' can be a dict or list; handle both:
                    if isinstance(series_data, dict):
                        series_data = [series_data]  # Wrap single dict in list

                    for s in series_data:
                        # Observations are typically in s["Obs"]
                        obs_list = s.get("Obs", [])
                        if isinstance(obs_list, dict):
                            obs_list = [obs_list]  # If single obs is a dict

                        for obs in obs_list:
                            time_period = obs.get("@TIME_PERIOD")
                            value = obs.get("@OBS_VALUE")

                            # Only store if value exists
                            if value is None:
                                continue

                            results.append({
                                "AREA_CODE": area_val,
                                "AREA_NAME": area_desc,
                                "SECTOR_CODE": sector_val,
                                "SECTOR_NAME": sector_desc,
                                "UNIT_CODE": unit_val,
                                "UNIT_NAME": unit_desc,
                                "INDICATOR": indicator,
                                "TIME_PERIOD": time_period,
                                "VALUE": value
                            })

                except KeyError:
                    # If the expected keys aren't there, skip
                    continue

###############################################################################
# 3. Convert to DataFrame
###############################################################################
df = pd.DataFrame(results)
print(f"\nRetrieved {len(df)} observations.")
print(df.head(10))

# Example: Save to CSV
df.to_csv("imf_gfsmab2017_data.csv", index=False)


Fetched 219 area codes.
Fetched 8 sector codes.
Fetched 2 unit codes.
Error retrieving CompactData/GFSMAB2017/A.DZ.S1314.XDC_R_B1GQ.G63N_FD4: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1321.XDC.G63N_FD4: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S13112.XDC_R_B1GQ.G33_F: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1313.XDC.GNLB__Z: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1314.XDC.G63N_FD4: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1314.XDC_R_B1GQ.G63N_FD4: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1312.XDC.G63N_FD4: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.AZ.S1312.XDC_R_B1GQ.GNLB__Z: Expecting value: line 1 column 1 (char 0)
Error retrieving CompactData/GFSMAB2017/A.BS.S1311

KeyboardInterrupt: 

In [19]:
import requests

# Base URL
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

# Function to query codelists
def fetch_codelist(codelist_key):
    try:
        response = requests.get(f'{url}{codelist_key}')
        response.raise_for_status()
        data = response.json()
        print(f"Codelist {codelist_key} retrieved successfully.")
        print("Sample data:", data)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for codelist {codelist_key}: {e}")
    except KeyError as e:
        print(f"KeyError: {e}. Check the response structure.")

# Dataset-specific codelists to test
codelists = [
    'CodeList/CL_FREQ',
    'CodeList/CL_AREA_GFSMAB2017',
    'CodeList/CL_SECTOR_GFSMAB2017',
    'CodeList/CL_UNIT_GFSMAB2017',
    'CodeList/CL_INDICATOR_GFSMAB2017'
]

# Fetch each codelist
for cl in codelists:
    print(f"\nFetching codelist: {cl}")
    fetch_codelist(cl)



Fetching codelist: CodeList/CL_FREQ
Codelist CodeList/CL_FREQ retrieved successfully.
Sample data: {'Error': 'Prognoz.SDMX.WebService.Exceptions.REST.RESTFaultException: Internal Server Error (Fault Detail is equal to There is an issue while retrieving the data, please check your query or contact an administrator. GKey = ae7bf460-5c9e-4f19-97d1-21f59b7173f9).'}

Fetching codelist: CodeList/CL_AREA_GFSMAB2017
Codelist CodeList/CL_AREA_GFSMAB2017 retrieved successfully.
Sample data: {'Structure': {'@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message', '@xsi:schemaLocation': 'http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure https://registry.sdmx.org/schemas/v2_0/SDMXStructure.xsd http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd', 'Header': {'ID': 'aae21dc1-9a11-4f2f-aded-512241cee121', 'Test

In [16]:
# Base URL for the IMF API
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  # Method to retrieve series information
search_term = 'Government Finance Statistics'  # Search term to find GSF

# Fetch the series list
response = requests.get(f'{url}{key}')
response.raise_for_status()  # Ensure the request was successful
series_list = response.json()['Structure']['Dataflows']['Dataflow']

# Extract data into a dictionary
data = []
gsf_ID = []
for series in series_list:
    if search_term in series['Name']['#text']:
        # Extract name and KeyFamilyID
        name = series['Name']['#text']
        key_family_id = series['KeyFamilyRef']['KeyFamilyID']
        
        # Add to list as a dictionary
        data.append({"name": name, "ID": key_family_id})
        gsf_ID.append(key_family_id)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save cleaned DataFrame as a CSV file
df.to_csv('government_finance_statistics.csv', index=False, header=True)

In [5]:
# Initialize an empty list to collect data
data = []

for ID in gsf_ID:
    key = f"DataStructure/{ID}"  # Method / series
    try:
        response = requests.get(f"{url}{key}")
        response.raise_for_status()  # Ensure the request was successful
        
        # Extract the list of dimensions
        dimension_list = response.json() \
            ['Structure']['KeyFamilies']['KeyFamily'] \
            ['Components']['Dimension']

        for n, dimension in enumerate(dimension_list):
            # Append the ID and dimension details to the data list
            data.append({"ID": ID, "Dimension_Number": n+1, "Codelist": dimension['@codelist']})
            print(f"{ID}: Dimension {n+1}: {dimension['@codelist']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed for {ID}: {e}")
    except KeyError as e:
        print(f"KeyError for {ID}: {e}")

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Optionally save the DataFrame to a file (CSV, Excel, etc.)
df.to_csv("dimension_results.csv", index=False)


GFSMAB2016: Dimension 1: CL_FREQ
GFSMAB2016: Dimension 2: CL_AREA_GFSMAB2016
GFSMAB2016: Dimension 3: CL_SECTOR_GFSMAB2016
GFSMAB2016: Dimension 4: CL_UNIT_GFSMAB2016
GFSMAB2016: Dimension 5: CL_INDICATOR_GFSMAB2016
GFSYFALCS2014: Dimension 1: CL_FREQ
GFSYFALCS2014: Dimension 2: CL_AREA_GFSYFALCS2014
GFSYFALCS2014: Dimension 3: CL_SECTOR_GFSYFALCS2014
GFSYFALCS2014: Dimension 4: CL_UNIT_GFSYFALCS2014
GFSYFALCS2014: Dimension 5: CL_INDICATOR_GFSYFALCS2014
GFSYFALCS2014: Dimension 6: CL_COUNTERPART_SECTOR_GFSYFALCS2014
GFSE2016: Dimension 1: CL_FREQ
GFSE2016: Dimension 2: CL_AREA_GFSE2016
GFSE2016: Dimension 3: CL_SECTOR_GFSE2016
GFSE2016: Dimension 4: CL_UNIT_GFSE2016
GFSE2016: Dimension 5: CL_INDICATOR_GFSE2016
GFSIBS2016: Dimension 1: CL_FREQ
GFSIBS2016: Dimension 2: CL_AREA_GFSIBS2016
GFSIBS2016: Dimension 3: CL_SECTOR_GFSIBS2016
GFSIBS2016: Dimension 4: CL_UNIT_GFSIBS2016
GFSIBS2016: Dimension 5: CL_GFS_STO_GFSIBS2016
GFSIBS2016: Dimension 6: CL_INSTR_ASSET_GFSIBS2016
GFSIBS2016: Di

TESTRUN GFSMAB2017

In [7]:
# Initialize an empty list to store the results
variable_data = []

test_df = df[df['ID']=='GFSMAB2017']
# Iterate through the unique codelists in the DataFrame
for codelist in test_df['Codelist'].unique():
    try:
        key = f"CodeList/{codelist}"
        response = requests.get(f"{url}{key}")
        response.raise_for_status()  # Ensure the request was successful

        # Extract codes and descriptions
        code_list = response.json()['Structure']['CodeLists']['CodeList']['Code']
        for code in code_list:
            variable_data.append({
                "Codelist": codelist,
                "Variable_Name": code['@value'],
                "Description": code['Description']['#text']
            })
            print(f"{codelist} - {code['Description']['#text']}: {code['@value']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed for codelist {codelist}: {e}")
    except KeyError as e:
        print(f"KeyError for codelist {codelist}: {e}")

# Convert the results into a DataFrame
variable_df = pd.DataFrame(variable_data)

# Display the resulting DataFrame
print(variable_df)

# Save the DataFrame to a file (optional)
variable_df.to_csv("variable_names_2017.csv", index=False)

KeyError for codelist CL_FREQ: 'Structure'
CL_AREA_GFSMAB2017 - Afghanistan: AF
CL_AREA_GFSMAB2017 - Albania: AL
CL_AREA_GFSMAB2017 - Algeria: DZ
CL_AREA_GFSMAB2017 - Angola: AO
CL_AREA_GFSMAB2017 - Anguilla: AI
CL_AREA_GFSMAB2017 - Antigua and Barbuda: AG
CL_AREA_GFSMAB2017 - Argentina: AR
CL_AREA_GFSMAB2017 - Armenia: AM
CL_AREA_GFSMAB2017 - Aruba: AW
CL_AREA_GFSMAB2017 - Australia: AU
CL_AREA_GFSMAB2017 - Austria: AT
CL_AREA_GFSMAB2017 - Azerbaijan: AZ
CL_AREA_GFSMAB2017 - Bahamas: BS
CL_AREA_GFSMAB2017 - Bahrain: BH
CL_AREA_GFSMAB2017 - Bangladesh: BD
CL_AREA_GFSMAB2017 - Barbados: BB
CL_AREA_GFSMAB2017 - Belarus: BY
CL_AREA_GFSMAB2017 - Belgium: BE
CL_AREA_GFSMAB2017 - Belize: BZ
CL_AREA_GFSMAB2017 - Benin: BJ
CL_AREA_GFSMAB2017 - Bhutan: BT
CL_AREA_GFSMAB2017 - Bolivia: BO
CL_AREA_GFSMAB2017 - Bosnia and Herzegovina: BA
CL_AREA_GFSMAB2017 - Botswana: BW
CL_AREA_GFSMAB2017 - Brazil: BR
CL_AREA_GFSMAB2017 - Brunei Darussalam: BN
CL_AREA_GFSMAB2017 - Bulgaria: BG
CL_AREA_GFSMAB2017 

In [ ]:
# Initialize an empty list to store the results
variable_data = []

# Iterate through the unique codelists in the DataFrame
for codelist in df['Codelist'].unique():
    try:
        key = f"CodeList/{codelist}"
        response = requests.get(f"{url}{key}")
        response.raise_for_status()  # Ensure the request was successful

        # Extract codes and descriptions
        code_list = response.json()['Structure']['CodeLists']['CodeList']['Code']
        for code in code_list:
            variable_data.append({
                "Codelist": codelist,
                "Variable_Name": code['@value'],
                "Description": code['Description']['#text']
            })
            print(f"{codelist} - {code['Description']['#text']}: {code['@value']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed for codelist {codelist}: {e}")
    except KeyError as e:
        print(f"KeyError for codelist {codelist}: {e}")

# Convert the results into a DataFrame
variable_df = pd.DataFrame(variable_data)

# Display the resulting DataFrame
print(variable_df)

# Save the DataFrame to a file (optional)
variable_df.to_csv("variable_names.csv", index=False)